In [1]:
import cv2
import skvideo
import math
import os
import numpy as np
cv2.__version__

'3.4.1'

In [2]:
import imageio

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
def video_to_folder_of_images(url, folder_name, period=1,  max_length=np.inf, mode ='gray', size=(224,224),
                            cut=True, (x1,x2, y1,y2) =(400, 1000, 120, 1000), verbose=False):
    """
    video to images folder by period and resize 224,224 and length = max_length
    if video seperated by black the function separate the video
    write images in folders in folder_name
    url is the name of video file
    cut is a flag of cutting the original image into x1, x2, y1, y2 square
    mode = {'gray', 'segment', 'lines', 'edges', 'diff', 'orb',  'filter'} is the way to preparation of images
    verbose is the flag of debarging
    """
    from skimage import filters
    video_count = 1
    image_count = 0
     # period chosen the image
    

    video = cv2.VideoCapture(url)
    while video.isOpened():
        success, image = video.read()
        
        # simply cut the most important image segment
        if cut:
            if np.any(image != None):
                image = image[x1:x2, y1:y2]
        
        # check the black seperating 
        black = np.all(image <= 0.01)
        if not black:
            # if not seperating creat the file to write in the same folder
            if not os.path.exists(folder_name +'/video_' + str(video_count).zfill(5)):
                os.makedirs(folder_name +'/video_' + str(video_count).zfill(5))
            # if yes seperating creat the new folder
            else:
                if len(os.listdir(folder_name +'/video_' + str(video_count).zfill(5))) >= max_length:
                    video_count += 1
                    os.makedirs(folder_name +'/video_' + str(video_count).zfill(5))
       
        if not success:
            print 'No read may be incorect the video file or the name, or the end of the video'
            break
        
        # seperate by black   
        if black:
            # if there is a black image turn to the next folder 
            if image_count != 0:
                video_count += 1
            image_count = 0
        else:
            if (image_count% period == 0): # use only images with the period
                
                filename = folder_name +'/video_' +str(video_count).zfill(5) + '/' \
                                  + str(image_count+1).zfill(5) +'_image.jpg'
                                
                # HSV represantation of the imade
                image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            
                # gray presantation of image
                if mode =='gray':
                    if np.any(image != None):
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                        #resize image
                        if size :
                            image = cv2.resize(image, size, interpolation=cv2.INTER_NEAREST)

                # find lines on the image and add lines to the image
                elif mode =='lines':
                    if np.any(image != None):
                        edges = cv2.Canny(image, 4, 70, apertureSize=3)
                        lines = cv2.HoughLines(edges, 1, np.pi/180, 200, 100, 10)
                    
                        for xx in lines:
                            rho, theta = xx[0]
                            a = np.cos(theta)
                            b = np.sin(theta)
                            x0 = a * rho
                            y0 = b * rho
                            x1 = int(x0 + 1000*(-b))
                            y1 = int(y0 + 1000* (a))
                            x2 = int(x0 - 1000*(-b))
                            y2 = int(y0 - 1000* (a))
                            
                            # use lines have only the angle
                            if (theta > 0.7) & (theta < 1.8):
                                image = cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
                            # resize image
                            if size :
                                image = cv2.resize(image, size, interpolation=cv2.INTER_NEAREST)
                
                # find contours on the image and turn the image in this style
                elif mode =='edges':
                    if np.any(image != None):
                        image = cv2.Canny(image, 60, 100, apertureSize=3)
                        #resize image
                        if size :
                            image = cv2.resize(image, size, interpolation=cv2.INTER_NEAREST)
                
                #find thresholds on the image and turn the image in this style    
                elif mode =='thresh':
                    if np.any(image != None):
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                        image = cv2.medianBlur(image, 5)
                        image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,
                                                      13, 4)
                        #resize image
                        if size :
                            image = cv2.resize(image, size, interpolation=cv2.INTER_NEAREST)
                
                #find background on the video and diff the backgrounf from the image  
                elif mode =='segment':
                    if np.any(image != None):
                        fgbg = cv2.createBackgroundSubtractorMOG2()
                        image = fgbg.apply(image)
                        #resize image
                        if size:
                            image = cv2.resize(image, size, interpolation=cv2.INTER_NEAREST)
                
                #find the difference between two serial images
                elif mode =='diff':
                    if np.any(image != None):
                        if image_count == 0:
                            image_old = image.copy()
                        image_diff = cv2.bitwise_xor(image, image_old)
                        image_old = image.copy()
                        image = cv2.cvtColor(image_diff, cv2.COLOR_BGR2GRAY)
                        #resize image
                        if size :
                            image = cv2.resize(image, size, interpolation=cv2.INTER_NEAREST)
                
                #find the org poins in images
                elif mode =='orb':
                    if np.any(image != None):
                         # Initiate STAR detector
                        orb = cv2.ORB_create(scoreType=cv2.ORB_FAST_SCORE)

                        # find the keypoints with ORB
                        kp = orb.detect(image,None)

                        # compute the descriptors with ORB
                        kp, des = orb.compute(image, kp)

                        # draw only keypoints location,not size and orientation
                        image = cv2.drawKeypoints(np.zeros_like(image),kp, None, color=(255,255,255), flags=cv2.DrawMatchesFlags_DEFAULT)
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                        #resize image
                        if size :
                            image = cv2.resize(image, size, interpolation=cv2.INTER_NEAREST)
                        
                #apply the filter to the images
                elif mode =='filter':
                    if np.any(image != None):
                        #filter parameters
                        blue_low = np.array([20, 30, 100])
                        blue_high = np.array([160, 250, 255])
                        # Threshold the HSV image to get only blue colors
                        mask = cv2.inRange(image, blue_low, blue_high)

                        # Bitwise-AND mask and original image
                        image = cv2.bitwise_and(image,image, mask= mask)
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                        #resize image
                        if size :
                            image = cv2.resize(image, size, interpolation=cv2.INTER_NEAREST)
                else:
                    print "Choce the mode from {'gray', 'segment', 'lines', 'edges', 'diff', 'orb',  'filter'}"
                
                if verbose:
                    print filename, '  ',image.shape
                
                # write the image in file   
                try:
                    imageio.imwrite(filename, image)
                except ValueError:
                    continue

            image_count +=1
    
    numbers_of_video = np.zeros(video_count, dtype=np.int32)
    
    # count the number of images that wrote in files in the folder
    for i,f in enumerate(os.listdir(folder_name)):
        numbers_of_video[i] = len(os.listdir(folder_name +'/' + f))
   
    print '*** Done *** write in ', folder_name
    print 'model of preprocession is, ', mode
    print 'number of videos ', len(numbers_of_video)
    print 'max_image_len ', int(numbers_of_video.max())
    print 'min_image_len ', int(numbers_of_video.min())
    return

In [17]:
folders_video = ['new_224_112_filter_1/video_good', 'new_224_112_filter_1/video_bad']
files_video = ['MVI_3906_1.mp4', 'MVI_3906.mp4']
for f, fd in zip(files_video, folders_video):
    video_to_folder_of_images(f, fd,  period=1, size=(224, 112) , mode='filter', verbose=True)

new_224_112_filter_1/video_good/video_00001/00001_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00002_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00003_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00004_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00005_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00006_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00007_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00008_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00009_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00010_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00011_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00012_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00013_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_

new_224_112_filter_1/video_good/video_00001/00113_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00114_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00115_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00116_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00117_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00118_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00119_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00120_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00121_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00122_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00123_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00124_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00125_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_

new_224_112_filter_1/video_good/video_00001/00225_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00226_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00227_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00228_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00229_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00230_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00231_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00232_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00233_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00234_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00235_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00236_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00237_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_

new_224_112_filter_1/video_good/video_00001/00337_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00338_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00339_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00340_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00341_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00342_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00343_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00344_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00345_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00346_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00347_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00348_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00349_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_

new_224_112_filter_1/video_good/video_00001/00449_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00450_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00451_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00452_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00453_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00454_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00455_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00456_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00457_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00458_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00459_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00460_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_00001/00461_image.jpg    (112, 224)
new_224_112_filter_1/video_good/video_

...

new_224_112_filter_1/video_bad/video_00002/04535_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04536_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04537_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04538_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04539_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04540_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04541_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04542_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04543_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04544_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04545_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04546_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04547_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04548_im

new_224_112_filter_1/video_bad/video_00002/04648_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04649_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04650_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04651_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04652_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04653_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04654_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04655_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04656_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04657_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04658_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04659_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04660_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04661_im

new_224_112_filter_1/video_bad/video_00002/04762_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04763_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04764_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04765_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04766_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04767_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04768_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04769_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04770_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04771_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04772_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04773_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04774_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04775_im

new_224_112_filter_1/video_bad/video_00002/04876_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04877_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04878_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04879_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04880_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04881_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04882_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04883_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04884_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04885_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04886_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04887_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04888_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04889_im

new_224_112_filter_1/video_bad/video_00002/04990_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04991_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04992_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04993_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04994_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04995_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04996_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04997_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04998_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/04999_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/05000_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/05001_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/05002_image.jpg    (112, 224)
new_224_112_filter_1/video_bad/video_00002/05003_im